# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-wit

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineerin

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

In [16]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/mistralai/Devstral-Small-2505',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/google/medgemma-4b-it',
 '/google/medgemma-27b-text-it',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces/google/rad_explain',
 '/spaces',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/ministere-culture/comparia-conversations',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/gajeshladhar/core-five',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [25]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'home page', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Team plan
Enterprise plan
Education plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support


In [34]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = "Είστε βοηθός που αναλύει το περιεχόμενο πολλών σχετικών σελίδων από έναν ιστότοπο της εταιρείας \
και δημιουργεί ένα σύντομο χιουμοριστικό, διασκεδαστικό, αστείο φυλλάδιο για την εταιρεία για υποψήφιους πελάτες, επενδυτές και υποψήφιους εργαζόμενους. Απαντήστε με μαρκάρισμα.\
Συμπεριλάβετε λεπτομέρειες για την εταιρική κουλτούρα, τους πελάτες και τις σταδιοδρομίες/θέσεις εργασίας, εάν έχετε τις πληροφορίες."



In [39]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company and in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nByteDance-Seed/BAGEL-7B-MoT\nUpdated\n4 days ago\n•\n3.09k\n•\n671\nmistralai/Devstral-Small-2505\nUpdated\nabout 8 hours ago\n•\n82.8k\n•\n558\ngoogle/gemma-3n-E4B-it-litert-preview\nUpdated\nabout 10 hours ago\n•\n497\ngoogle/medgemma-4b-it\nUpdated\n5 days ago\n•\n12.6k\n•\n232\ngoogle/medgemma-27b-text-it\nUpdated\nabout 8 hours ago\n•\n6.18k\n•\n165\nBrowse 1M+ models\nSpaces\nRunning

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [42]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Welcome to Anthropic: We Make AI Feel Like a Walk in the Park!

## Who Are We?  
At Anthropic, we're on a mission to build AI systems that serve humanity's long-term interests. Think of us as the friendly neighborhood AI creators, but instead of capes, we wear lab coats and talk to Claude! Yes, Claude—our AI buddy who's always ready to chat, lend a hand, or assist in your next big idea!

## Meet Claude: Your AI Sidekick  
- **Claude Opus 4 & Sonnet 4**: These aren’t just fancy names; they're our most powerful AI models designed for coding, researching, and writing. Imagine having a super-smart friend who can whip up a poem, crack a code, and even help you with your homework—all at once!  
- **Typing Avenger**: Code with Claude and watch your programming woes vanish. No more "Oops, did I just delete my entire project?" moments!

## Our Corporate Culture: Chill Vibes, Serious Impact  
- **Safety First**: While we explore cutting-edge AI tech, we act like responsible adult children—keeping tabs on our imagination. We focus on responsible AI development and ensure our innovations prioritize the human experience.  
- **Learning is Fun**: Thanks to **Anthropic Academy**, we ensure our team grows smarter and more skilled—because who doesn’t enjoy a good learning curve?  

## Why People Love Us  
From education to enterprise solutions, we help people build effective AI agents. Our users come to us not just for the tech, but because we make their lives easier (and a bit more enjoyable). Here’s what our fans say:  
> “Thanks to Claude, I finally finished my screenplay... in one weekend!”  
> — Aspiring screenwriter with an enthusiasm for AI!

## Join Our Team: A Place for Dreamers and Doers  
We're not only building the future of AI; we're building a fun workplace for you too!  
- **Open Roles**: Want to help shape the future of AI? Check our careers page!  
- **Bring Your Ideas**: Have a revolutionary thought? We’re all ears (and algorithms)!

---

### Let’s Chat!  
Want to explore how Claude can transform your work or learn more about us?  
- **Contact Sales**: We’re more than happy to help you figure out the magic our tech can bring to your business.  
- **Try Claude**: Access our demo, and don’t be surprised if Claude ends up being your new best friend!

**Anthropic**: Where AI Meets Humanity—In the Most Fun Way Possible!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [28]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}



# 🌟 Welcome to Anthropic! 🌟

At Anthropic, AI isn’t just part of our culture; it's the *heart* of it! 🚀✨ Whether you're a potential customer, an eager investor, or a curious job seeker, there's something here for you that *might just* make you smile (or laugh out loud). 🤪

## What is Anthropic? 🧠🤖

We're probably the most fun you can have while chatting with an AI! Meet **Claude**, our latest and greatest AI companion—think of him as the friend who knows *a little bit* about *everything* (but way more polite). From coding to researching, or just answering those pesky "why is the sky blue?" questions, Claude has your back!

### Meet the Claude Family: 🌈

- **Claude Opus 4**: The grand maestro of AI, capable of conducting all your tech needs with finesse. 🎶
- **Claude Sonnet 4**: Poetic and expressive; a creative soul ready to write everything from sonnets to *that* email you’ve been avoiding. ✍️
- **Claude Haiku 3.5**: The minimalist; sometimes less is more! 🥷

## We're Building Tools for Humanity! 🌍❤️

At Anthropic, we're urban designers of the AI world with a heart. We focus on developing tools like Claude that put **human benefit** at their foundation. We believe that AI can make the planet a better place—after all, who doesn’t want a personal assistant that understands human needs? 

### Our Promise: 🌱

We’re committed to responsible scaling of technology and ethical AI development. We even crafted an *“Anthropic Academy”*—where you can learn to build with Claude while having a great time! It’s like Hogwarts for tech enthusiasts. 🧙‍♂️💻

## Join Our Dream Team! 🤩

We're always on the lookout for passionate folks to help us shape the future of AI. Think you've got what it takes? Here’s what you can look forward to:

- **Career Growth**: You’re not just a cog in the machine; you’re the *whole* machine! 🚀
- **Fun Culture**: Our workplace resembles more of a brainstorming session than a meeting, complete with collaborative vibes and a sprinkle of humor. 🌈
- **Diversity & Inclusion**: We celebrate uniqueness like it’s an Olympic sport! 🏅

Check out our open roles and see how you can jump onto the Claude express! All aboard? 🚂💨

## What Our Clients Say: 🗣️

- "Claude has made my life 10x easier!" – A relieved user.
- "I can't believe I once typed my own emails!" – A former non-believer in AI.

So, whoever you are, and however you found us, remember this: at Anthropic, we make AI friendly, responsible, and oh-so-fun! 🌟

---

Interested? Let’s chat! Try out Claude or follow us on our journey to revolutionize AI. Your future self (and Claude!) will thank you! 🎉


This brochure introduces Anthropic with a mix of humor and information, highlighting their AI tools, cultural values, and career opportunities.

In [29]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}]}



# Welcome to Anthropic

At **Anthropic**, we are dedicated to building advanced AI technology that prioritizes human safety and well-being. Our flagship product, **Claude**, offers various AI models designed to assist users across different sectors, including coding, researching, and customer support.

## Our Mission

Our mission is simple yet profound: to create AI tools that enrich human experiences while maintaining a commitment to responsible innovation. We believe in not just advancing technology but doing so in a way that considers its implications on society.

## Our Products

- **Claude Models**: Explore our family of models, including Claude Opus 4 and Claude Sonnet 4, which are tailored for coding, research, and writing.
- **API Solutions**: Build and integrate powerful AI agents into your applications with our comprehensive API offerings.
- **Claude Apps**: Download our intuitive apps to enhance your productivity with Claude by your side.

## Company Culture

At Anthropic, our culture fosters collaboration, transparency, and continuous learning. We embrace bold steps in AI development while maintaining thoughtful practices around its impacts on society. Our commitment to responsible AI is evident in our **Responsible Scaling Policy** and our efforts in **AI Safety**.

### Learn with Us

Join the **Anthropic Academy**, where you can access resources and learn to build effectively with Claude. We believe in empowering our community to harness AI responsibly.

## Customers

We cater to a diverse set of customers, from individual developers to large enterprises. Our solutions are designed to meet the needs of various sectors, helping them leverage AI for enhanced productivity and innovation.

## Careers at Anthropic

We are always on the lookout for passionate individuals who want to contribute to the future of AI. If you're interested in joining a team that values responsibility and transparency in technology, explore our **open roles** and become a part of our mission to build a better future.

### Connect with Us

Interested in our offerings or want to learn more? Feel free to reach out, or try Claude today!

**Website**: [Anthropic](https://www.anthropic.com)  
**Follow Us**: Stay tuned for updates by visiting our news section.  
**Careers Page**: [Join Our Team](https://www.anthropic.com/careers)

---
*“If knowledge is power and we're building machines that have more knowledge than us, what will happen between us and the machines?” - Deep Ganguli, Research Lead, Societal Impacts*



In [33]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face! 
## Your Friendly Neighborhood AI Superheroes!

**Are you ready to join the AI revolution?** At Hugging Face, we’re not just building the future – we’re cuddling it. Our community is here to support you with all your machine learning dreams, datasets, and models (yes, hugs are included!).

## 🤖 What We Do

- **Models**: We have over **1 MILLION models** for you to explore! You could say our collection is *out of this world*. (Seriously, check out our *Astronomically Awesome AI Models* section.)
  
- **Datasets**: Looking for that perfect dataset? You’re in luck! We’ve got **250,000+ datasets** ready for your perusal. You could say it’s like a buffet, but only for data enthusiasts!

- **Spaces**: Create and host your own applications with our **Spaces**! It's like renting an apartment, but instead of neighbors, you get to hang out with some very intelligent machines. 

## 🦄 Why Hugging Face? 

We’re not just a bunch of tech nerds; we’re a community! Our platform is designed to help you collaborate and create phenomenal AI applications. Think of us as your trusty sidekick in your quest for machine learning greatness.

## 🎉 Company Culture

At Hugging Face, we believe in the power of collaboration. Picture this: a team of enthusiastic AI aficionados working together in harmony, fueled by cups of coffee and a shared love for all things neural networks. Our work culture is open, friendly, and we promise there are no scary algorithms lurking in the corner (well, maybe just a few!).

## 📈 Our Customers

Over **50,000 organizations** are embracing the AI magic with us, including tech giants like Google, Microsoft, and Amazon. That’s right, we’re the cool kids on the block, and our friends are pretty famous!

## 🚀 Careers

Interested in joining our wonderful Hugging Face family? We’re hunting for talents who can help us redefine the bounds of machine learning. Whether you’re a coding wizard or a data conjurer, we want you!

### Perks of Working at Hugging Face:
- Flexible hours (because life is not just work)
- Unlimited snacks (okay, we might have a *tiny* amount of obsession with snacks)
- Opportunity to be part of a groundbreaking AI community

So whether you’re an AI guru or just someone who likes hugging their computer, come join us at Hugging Face! 🌟

## Get Started Today!

**Visit us at [huggingface.co](https://huggingface.co)**

Remember: A day without a hug is like a day without sunshine!



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [ ]:
stream_brochure("Anthropic", "https://anthropic.com")